In [1]:
# Importing libraries

import pandas as pd
import os
import shutil
import sys

In [2]:
# loading scripts
sys.path.insert(1, '../scripts')
sys.path.append("..")
sys.path.append(".")

from data_cleaning import DataCleaner
from deep_learning_m import Separating

DC = DataCleaner("../logs/preprocessing_notebook.log")

In [3]:
source_folder = "../data/train_new/"
destination_folder_train = '../data/train_model/'
destination_folder_valid = '../data/valid_model/'

Separating.train_valid(source_folder,destination_folder_train,destination_folder_valid)

In [ ]:
len(os.listdir('../data/train_model')),len(os.listdir('../data/valid_model'))